In [1]:
#================================================================================================
# 동적 양자화(Dynamic Quantization) 테스트 
# => 모델의 파라미터를 lower bit로 표현함으로서 계산과 메모리 access 속도를 높이는 경량화 기법
# => 보통 32비트 부동소수점 연산을 8비트 정수로 변환하는 방식 사용
# => 동적 양자화, 정적 양자화, 학습을 통한 양자화 Training 등이 있음.
# => 아래예재는 이미 만들어진 bert 모델을 가지고 동적 양자화 하는 예제임
#
# [동적 양자화 torch 함수]
# quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
#
# 참고자료 : 
# https://velog.io/@jooh95/%EB%94%A5%EB%9F%AC%EB%8B%9D-Quantization%EC%96%91%EC%9E%90%ED%99%94-%EC%A0%95%EB%A6%AC
# https://tutorials.pytorch.kr/intermediate/dynamic_quantization_bert_tutorial.html
# https://huggingface.co/docs/optimum/onnxruntime/modeling_ort
#
# 참고 소스 :
# https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/dynamic_quantization_bert_tutorial.ipynb#scrollTo=vnMpkij7IgN7
#
# [테스트시]
# => torch를 이용한 동적 양자화 기법은 추론시 cpu만 지원함.(*따라서 GPU 서버에서는 사용못하고, 다른 CPU 서버에서는 적용해볼만함)
# => 양자화된 모델을 저장후 huggingface 모델에서 불러와서 추론해보면 정확도가 기본으로 떨어짐.
#    단 원본 모델을 양자화 모델로 변환후 저장하지 않고 추론시에는 정확도가 약간 떨어짐.
# => 확실히 용량은 819MB->574MB 정도로 약 70% 정도 줌. 
#================================================================================================

import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

from os import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging
from tqdm.notebook import tqdm
import os

logger = mlogging(loggername="bertnlitest", logfilename="bertnlitest")
device = GPU_info()
seed_everything(111)

logfilepath:../../log/bwdataset_2022-05-25.log
logfilepath:../../log/qnadataset_2022-05-25.log
logfilepath:bertnlitest_2022-05-25.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#############################################################################################

model_path = '../../model/classification/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327-ft-nli-0328'
vocab_path = '../../model/classification/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327-ft-nli-0328'

# tokeniaer 및 model 설정
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=False)
# strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
# do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)
#tokenizer = BertTokenizer(vocab_file=vocab_path, strip_accents=False, do_lower_case=False) 
                       
# 레벨을 1개만 선택하는 경우
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
#model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)

# 레벨을 멀티로 선택해야 하는 경우
#model = BertForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification",num_labels=6)


In [3]:
# 양자화 처리 => Linear 레이어에 대해 int8 로 양자화 처리
# => 원본 모델은 gpu 설정하면 안됨
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(167550, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.1, inplace=False)
            )
     

In [ ]:
'''
# 모델 저장
# => torch.save 로 저장하는 방법 
from transformers import WEIGHTS_NAME, CONFIG_NAME

output_dir = "./quantized/"
os.makedirs(output_dir, exist_ok=True)

output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
print(output_model_file)
print(output_config_file)

torch.save(quantized_model.state_dict(), output_model_file)
quantized_model.config.to_json_file(output_config_file)
tokenizer.save_pretrained(output_dir)
'''

In [ ]:
'''
# 모델 저장
# => huggingface save_pretrained 로 저장
quantized_output_dir = "./quantized"
os.makedirs(quantized_output_dir, exist_ok=True)
quantized_model.save_pretrained(quantized_output_dir)
tokenizer.save_pretrained(quantized_output_dir)
'''

In [4]:
# 평가 data loader 생성
from os import sys
sys.path.append('..')
from myutils import ClassificationDataset, KlueNLICorpus, data_collator
from torch.utils.data import DataLoader, RandomSampler

#############################################################################
# 변수 설정
#############################################################################
max_seq_len = 128   # 글자 최대 토큰 길이 해당 토큰 길이 이상은 잘린다.
batch_size = 32        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = True   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
#############################################################################

# corpus 파일 설정
corpus = KlueNLICorpus()

# 평가 dataset 생성
file_fpath = '../../korpora/klue-nli/klue-nli-v1.1_dev.json'
dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)

# 평가 dataloader 생성
eval_loader = DataLoader(dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)

print('eval_loader_len: {}'.format(len(eval_loader)))

2022-05-26 08:53:59,096 - bwpdataset - INFO - Creating features from dataset file at ../../korpora/klue-nli/klue-nli-v1.1_dev.json
2022-05-26 08:53:59,098 - bwpdataset - INFO - loading data... LOOKING AT ../../korpora/klue-nli/klue-nli-v1.1_dev.json
2022-05-26 08:53:59,144 - bwpdataset - INFO - tokenize sentences, it could take a lot of time...
2022-05-26 08:54:00,188 - bwpdataset - INFO - tokenize sentences [took 1.043 s]


  0%|          | 0/3000 [00:00<?, ?it/s]

2022-05-26 08:54:00,216 - bwpdataset - INFO - *** Example ***
2022-05-26 08:54:00,217 - bwpdataset - INFO - sentence A, B: 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다. + 어떤 방에서도 흡연은 금지됩니다.
2022-05-26 08:54:00,217 - bwpdataset - INFO - tokens: [CLS] 흡연자 ##분 ##들은 발코니 ##가 있는 방이 ##면 발코니 ##에서 흡연 ##이 가능 ##합 ##니다 . [SEP] 어떤 방 ##에서 ##도 흡연 ##은 [UNK] . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
2022-05-26 08:54:00,218 - bwpdataset - INFO - label:

eval_loader_len: 94


In [5]:
import time

# 평가 시작
quantized_model.eval()
#model.eval()

total_loss = 0
total_len = 0
total_correct = 0

start = time.time()
logger.info(f'---------------------------------------------------------')

for data in tqdm(eval_loader):
     # 입력 값 설정
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']   
    labels = data['labels']
 
    # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
    # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
    # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
    with torch.no_grad():
        # 모델 실행
        outputs = quantized_model(input_ids=input_ids, 
                       attention_mask=attention_mask,
                       token_type_ids=None,
                       labels=labels)
    
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)

logger.info(f"eval-accuracy: {total_correct / total_len}")
logger.info(f'---------------------------------------------------------')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-05-26 08:54:02,779 - bertnlitest - INFO - ---------------------------------------------------------


  0%|          | 0/94 [00:00<?, ?it/s]

/tmp/ipykernel_120848/1588122005.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)
2022-05-26 08:56:06,551 - bertnlitest - INFO - eval-accuracy: 0.685
2022-05-26 08:56:06,552 - bertnlitest - INFO - ---------------------------------------------------------
2022-05-26 08:56:06,553 - bertnlitest - INFO - === 처리시간: 123.774 초 ===
2022-05-26 08:56:06,553 - bertnlitest - INFO - -END-

